In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/janatahack-independence-day-2020-ml-hackathon/sample_submission_UVKGLZE.csv
/kaggle/input/janatahack-independence-day-2020-ml-hackathon/train.csv
/kaggle/input/janatahack-independence-day-2020-ml-hackathon/test.csv


In [67]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import hamming_loss
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.preprocessing import MultiLabelBinarizer

In [68]:
train=pd.read_csv('/kaggle/input/janatahack-independence-day-2020-ml-hackathon/train.csv')
test=pd.read_csv('/kaggle/input/janatahack-independence-day-2020-ml-hackathon/test.csv')
sub=pd.read_csv('/kaggle/input/janatahack-independence-day-2020-ml-hackathon/sample_submission_UVKGLZE.csv')

In [69]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20972 entries, 0 to 20971
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    20972 non-null  int64 
 1   TITLE                 20972 non-null  object
 2   ABSTRACT              20972 non-null  object
 3   Computer Science      20972 non-null  int64 
 4   Physics               20972 non-null  int64 
 5   Mathematics           20972 non-null  int64 
 6   Statistics            20972 non-null  int64 
 7   Quantitative Biology  20972 non-null  int64 
 8   Quantitative Finance  20972 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 1.4+ MB


In [70]:
sub.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,0,0,0
1,20974,0,0,0,0,0,0
2,20975,0,0,0,0,0,0
3,20976,0,0,0,0,0,0
4,20977,0,0,0,0,0,0


In [71]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20972 entries, 0 to 20971
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    20972 non-null  int64 
 1   TITLE                 20972 non-null  object
 2   ABSTRACT              20972 non-null  object
 3   Computer Science      20972 non-null  int64 
 4   Physics               20972 non-null  int64 
 5   Mathematics           20972 non-null  int64 
 6   Statistics            20972 non-null  int64 
 7   Quantitative Biology  20972 non-null  int64 
 8   Quantitative Finance  20972 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 1.4+ MB


In [72]:
train.columns

Index(['ID', 'TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'],
      dtype='object')

In [73]:
train["com"] = train["TITLE"]+train["ABSTRACT"]
train.drop(labels = ["ID","TITLE","ABSTRACT"],axis=1,inplace = True)
# train.head()

In [74]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8989 entries, 0 to 8988
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        8989 non-null   int64 
 1   TITLE     8989 non-null   object
 2   ABSTRACT  8989 non-null   object
dtypes: int64(1), object(2)
memory usage: 210.8+ KB


In [75]:
train['Computer Science'].value_counts()
train['Physics'].value_counts()
train['Mathematics'].value_counts()
train['Statistics'].value_counts()
train['Quantitative Biology'].value_counts()
train['Quantitative Finance'].value_counts()


0    12378
1     8594
Name: Computer Science, dtype: int64

0    14959
1     6013
Name: Physics, dtype: int64

0    15354
1     5618
Name: Mathematics, dtype: int64

0    15766
1     5206
Name: Statistics, dtype: int64

0    20385
1      587
Name: Quantitative Biology, dtype: int64

0    20723
1      249
Name: Quantitative Finance, dtype: int64

In [76]:
all_words = list(train['com'].str.lower().str.split(' ', expand=True).stack().unique())
all_words

['reconstructing',
 'subject-specific',
 'effect',
 'maps',
 '',
 'predictive',
 'models',
 'allow',
 'inference',
 'when',
 'analyzing',
 'disease\nrelated',
 'alterations',
 'in',
 'neuroimaging',
 'data.',
 'given',
 'a',
 "subject's",
 'data,',
 'can\nbe',
 'made',
 'at',
 'two',
 'levels:',
 'global,',
 'i.e.',
 'identifiying',
 'condition',
 'presence',
 'for',
 'the\nsubject,',
 'and',
 'local,',
 'detecting',
 'on',
 'each',
 'individual\nmeasurement',
 'extracted',
 'from',
 'the',
 'while',
 'global',
 'is',
 'widely\nused,',
 'local',
 'inference,',
 'which',
 'can',
 'be',
 'used',
 'to',
 'form',
 'maps,\nis',
 'rarely',
 'because',
 'existing',
 'often',
 'yield',
 'noisy',
 'detections',
 'composed',
 'of\ndispersed',
 'isolated',
 'islands.',
 'this',
 'article,',
 'we',
 'propose',
 'reconstruction\nmethod,',
 'named',
 'rsm,',
 'improve',
 'of',
 'predictive\nmodeling',
 'approaches',
 'particular,',
 'binary',
 'classifiers.',
 'rsm',
 'specifically\naims',
 'reduce'

In [77]:
# train['train_flag'] = 1
# test['train_flag'] = 0
# df=pd.concat((train,test))
# df_train, df_test = df_data[df_data.train_flag == 1], df_data[df_data.train_flag == 0]

In [78]:
text=train.com[0].lower()
text

"reconstructing subject-specific effect maps  predictive models allow subject-specific inference when analyzing disease\nrelated alterations in neuroimaging data. given a subject's data, inference can\nbe made at two levels: global, i.e. identifiying condition presence for the\nsubject, and local, i.e. detecting condition effect on each individual\nmeasurement extracted from the subject's data. while global inference is widely\nused, local inference, which can be used to form subject-specific effect maps,\nis rarely used because existing models often yield noisy detections composed of\ndispersed isolated islands. in this article, we propose a reconstruction\nmethod, named rsm, to improve subject-specific detections of predictive\nmodeling approaches and in particular, binary classifiers. rsm specifically\naims to reduce noise due to sampling error associated with using a finite\nsample of examples to train classifiers. the proposed method is a wrapper-type\nalgorithm that can be used w

> Here Y is multilabelbinarized.... how can we reverse back from here if we want 

In [79]:
stemmer = SnowballStemmer("english")
wordlem = WordNetLemmatizer()
corpus=[]
STOPWORDS = set(stopwords.words('english'))
for i in range(0,len(train.com)):
    text = train.com[i].lower()# lowercase text
    text=text.replace(r'\d+(\.\d+)?', 'numbr')
    text = re.sub("[!@#$+%*:'-_=./(){}\[\]\|@,;]", ' ', text)
    text=re.sub(r"\d+", "", text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    wordList = nltk.word_tokenize(text)
    words=[wordlem.lemmatize(word) for word in wordList]
    stemWords = [stemmer.stem(word) for word in words]
    sent=' '.join(stemWords)
#     s=sent.apply(lambda x: word_tokenize(x))
    corpus.append(sent)# delete stopwors from text
#     corpus.append(text)
    
    

In [80]:
corpus

['reconstruct subject specif effect map predict model allow subject specif infer analyz diseas relat alter neuroimag data given subject data infer made two level global e identifiy condit presenc subject local e detect condit effect individu measur extract subject data global infer wide use local infer use form subject specif effect map rare use exist model often yield noisi detect compos dispers isol island articl propos reconstruct method name rsm improv subject specif detect predict model approach particular binari classifi rsm specif aim reduc nois due sampl error associ use finit sampl exampl train classifi propos method wrapper type algorithm use differ binari classifi diagnost manner e without inform condit presenc reconstruct pose maximum posteriori problem prior model whose paramet estim train data classifi specif fashion experiment evalu perform synthet generat data data alzheim diseas neuroimag initi adni databas result synthet data demonstr use rsm yield higher detect accur

In [81]:
x=corpus
# y = train.iloc[:, 3:] 

In [82]:
target_cols = ['Computer Science', 'Physics', 'Mathematics','Statistics', 'Quantitative Biology', 'Quantitative Finance']
y = train[target_cols]

In [83]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=13)

In [84]:

# X_train.shape
y_train.shape
# X_test.shape
y_test.shape

(16777, 6)

(4195, 6)

In [85]:
word_vectorizer = TfidfVectorizer(
    stop_words='english',
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{2,}',  #vectorize 2-character words or more
    ngram_range=(1, 1),
    max_features=30000)

# fit and transform on it the training features
word_vectorizer.fit(X_train)
X_train_word_features = word_vectorizer.transform(X_train)

TfidfVectorizer(max_features=30000, stop_words='english',
                strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{2,}')

In [86]:
test_features=word_vectorizer.transform(X_test)

In [92]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

classifier=OneVsRestClassifier(LinearSVC(penalty="l2",loss='hinge'), n_jobs=-1)
classifier.fit(X_train_word_features,y_train)
predictions=classifier.predict(test_features)
print(f1_score(y_test,predictions, average='micro'))
# sgd = SGDClassifier(loss='log', max_iter=100, random_state=101, class_weight='balanced')
# ovr = OneVsRestClassifier(sgd)
# ovr.fit(X_train_word_features, y_train)
# y_pred_class = ovr.predict(test_features)
# print(f1_score(y_test,y_pred_class, average='micro'))


/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


OneVsRestClassifier(estimator=LGBMClassifier(n_estimators=300), n_jobs=-1)

0.8034120992254142


In [ ]:

# lp_classifier = LabelPowerset(LogisticRegression(max_iter=1000))
# lp_classifier.fit(X_train_word_features, y_train)
# lp_predictions = lp_classifier.predict(test_features)
# print("Accuracy = ",accuracy_score(y_test,lp_predictions))
# print("F1 score = ",f1_score(y_test,lp_predictions, average="micro"))
# print("Hamming loss = ",hamming_loss(y_test,lp_predictions))

In [ ]:
t_f=test['TITLE']+test["ABSTRACT"]

In [ ]:
t_ff=[]
for i in range(0,len(t_f)):
    text = t_f[i].lower()# lowercase text
    text=text.replace(r'\d+(\.\d+)?', 'numbr')
    text = re.sub("[!@#$+%*:'-_=./(){}\[\]\|@,;]", ' ', text)
    text=re.sub(r"\d+", "", text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    wordList = nltk.word_tokenize(text)
    words=[wordlem.lemmatize(word) for word in wordList]
    stemWords = [stemmer.stem(word) for word in wordList]
    sent=' '.join(stemWords)
#     s=sent.apply(lambda x: word_tokenize(x))
    t_ff.append(sent)

In [ ]:
final_features = word_vectorizer.transform(t_ff)

In [ ]:
predictions=classifier.predict(final_features)
# prediction1=ovr.predict(final_features)

In [ ]:
submission=pd.DataFrame(predictions, columns=['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance'])
submission=pd.concat([test['ID'],submission],axis=1)
submission.to_csv("topic2.csv", index=False)
print(submission.shape)
submission.head()

In [ ]:
# final = lp_classifier.predict(final_features)


In [ ]:
# type(lp_predictions)

In [ ]:
# d = pd.DataFrame(final.todense())

In [ ]:
# d.columns = ['a', 'b','c','d','e','f']

In [ ]:
# submit=pd.DataFrame({'ID':                  test.ID,
#                      'Computer Science':    d['a'],
#                      'Physics':             d['b'],
#                      'Mathematics':         d['c'],
#                      'Statistics':          d['d'],
#                      'Quantitative Biology':d['e'], 
#                      'Quantitative Finance':d['f'],})

In [ ]:
# submit.to_csv('sub.csv',index=False)